In [49]:
import sys 
sys.path.append('/mnt/data/graph')
from graph_editing import MedicalGraphManager
from data_loader import CheXpertGraphDataset
import pickle
import numpy as np

In [52]:
ds = CheXpertGraphDataset(
    csv_path= "/mnt/data/diffusion_graph/reports_processed.csv",
    embeddings_path = "/mnt/data/diffusion_graph/new_embeddings_768.pkl",
    load_graphs = True,
    embedding_dim="original",
    graphs_path = "/mnt/data/diffusion_graph/reports_processed_graphs.pkl",
)

Loading CheXpertGraphDataset (split=train, embedding_dim=original)...


  Loaded CSV: 223462 total samples
  After train filter: 223228 samples
  Opening embeddings: /mnt/data/diffusion_graph/new_embeddings_768.pkl
  Pickle loaded: (223462, 768)
  Loading graphs: /mnt/data/diffusion_graph/reports_processed_graphs.pkl
  Loaded 223462 graphs
✓ Dataset ready: 223228 samples


In [53]:
# Initialize manager
manager = MedicalGraphManager(ds.graphs)

In [54]:
nodes = manager.get_all_nodes()
edges = manager.get_all_edges()

In [55]:
g = manager.edit_instance(1,add_nodes=["cardiomegaly"])

In [31]:
from graph_embedding import SimpleGraph2VecEmbedder
embedder = pickle.load(open('/mnt/data/diffusion_graph/graph2vec_embeddings.pkl', 'rb'))

In [56]:
def check_emb_cosine(idx):
    emb_orig = embedder.encode(manager.graphs[idx])
    if "cardio" in manager.graphs[idx].nodes():
        edited = manager.edit_instance(idx, remove_nodes=["cardio"])
    else:
        edited = manager.edit_instance(idx, add_nodes=["cardio"])
    emb_edited = embedder.encode(edited)
    random_idx = np.random.randint(0, len(graphs))
    emb_random = embedder.encode(graphs[random_idx])
    edited_cosine = np.dot(np.squeeze(emb_edited), np.squeeze(emb_orig))/(np.linalg.norm(emb_edited)*np.linalg.norm(emb_orig))
    random_cosine = np.dot(np.squeeze(emb_edited), np.squeeze(emb_random))/(np.linalg.norm(emb_edited)*np.linalg.norm(emb_random))
    return edited_cosine, random_cosine

edited_cosines = []
random_cosines = []
for x in range(1000):
    edited_cosine, random_cosine = check_emb_cosine(x)
    edited_cosines.append(edited_cosine)
    random_cosines.append(random_cosine)

print(np.mean(edited_cosines))
print(np.mean(random_cosines))


0.92200637
0.39908257
